In [1]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator
from keras import backend
import os
 
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n".format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1
 
    def on_train_begin(self, logs={}):
        self.losses += 'Training begins...\n'

script_dir = os.getcwd()
dataset_path = os.path.join(script_dir, 'dataset')
train_path = os.path.join(script_dir, 'train')
# validation_path = os.path.join(script_dir, 'validation')
test_path = os.path.join(script_dir, 'test')

Using TensorFlow backend.


# Split dataset into train/validation/test

In [5]:
# use this one time

def check_folder_split_exist(path):
    print(path)
    if os.path.exists(path):
        return True, 'path exist!'
    else:
        os.makedirs(path)
        return False, 'path created!'

[check_folder_split_exist(path) for path in [train_path, test_path]]

/home/robomex/Desktop/Thapar/sign-language-detection/train/train
/home/robomex/Desktop/Thapar/sign-language-detection/train/test


[(False, 'path created!'), (False, 'path created!')]

In [6]:
# Initialising the CNN
classifier = Sequential()
 
# Step 1 - Convolution
input_size = (128, 128)
classifier.add(Conv2D(32, (3, 3), input_shape=(*input_size, 3), activation='relu'))
 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))  # 2x2 is optimal
 
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=1, activation='sigmoid'))
 
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 
# Part 2 - Fitting the CNN to the images
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 class_mode='binary')
 
test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            class_mode='binary')
 
# Create a loss history
history = LossHistory()
 
r= classifier.fit_generator(training_set,
                         steps_per_epoch=8000/batch_size,
                         epochs=15,
                         validation_data=test_set,
                         validation_steps=2000/batch_size,
                         workers=12,
                         #max_q_size=100,
                         callbacks=[history])


# Save model
model_backup_path = os.path.join(script_dir, 'jaya_smriti.h5')
classifier.save(model_backup_path)
print("Model saved to", model_backup_path)
 
# Save loss history to file
loss_history_path = os.path.join(script_dir, 'loss_history.log')
myFile = open(loss_history_path, 'w+')
myFile.write(history.losses)
myFile.close()
 
backend.clear_session()
print("The model class indices are:", training_set.class_indices)

Found 12888 images belonging to 2 classes.
Found 3228 images belonging to 2 classes.
Epoch 1/15
250/250 [==============================] - 68s 271ms/step - loss: 0.6870 - accuracy: 0.5408 - val_loss: 0.5948 - val_accuracy: 0.6558


TypeError: unsupported format string passed to NoneType.__format__